### 0. Install libaries

I need the following packages. In case, just install them!

In [ ]:
#!conda install -c conda-forge beautifulsoup4 --yes
#!conda install -c conda-forge requests --yes 

### 1. Transform data from wiki into a dataframe

Call the wiki-html and find all tables in html-code:

In [ ]:
from bs4 import BeautifulSoup
import requests

# get html of the url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)
results.text

# find tables in wiki-html
soup = BeautifulSoup( results.text, 'html.parser')
tables = soup.find_all('table', class_='sortable')
print ('Found {} table(s) in the extracted wiki-html.'.format( len (tables ) ) )

Scrap the of the wiki-html-table headings:

In [ ]:
ths = tables[0].find_all('th')  # find all headings in table 
headings = [th.text.strip() for th in ths]
headings

Rename one column:

In [ ]:
headings[0] =  'Postalcode'

headings

Scrap the content of the wiki-html-table:

In [ ]:
rows = []

for tr in tables[0].find_all('tr'): # find all rows in table
    tds = tr.find_all('td')
    if not tds: # if NOT table data, for example the headings of the table
        continue
    row = [td.text.strip() for td in tds[:3]]
    rows.append(row)
rows 

Build Dataframe from extracted headings and the data-array "rows" as the list of lists:

In [ ]:
import pandas as pd
Canada = pd.DataFrame(rows, columns=headings)

Canada

# To shorten the scraping process one can use:
# Canada = pd.read_html(url, header=0)[0]

### 2. Clean Dataframe

**Task:** *Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.*

Drop rows where column "Borough" has the value "Not assigned":

In [ ]:
import numpy as np

Canada.replace(to_replace='Not assigned', value=np.NaN, inplace=True)
Canada.dropna(axis=0, subset=['Borough'], inplace=True)
Canada.reset_index(drop = True, inplace = True)
Canada

**Task:** *If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.*

Replace empty fields of Neighbourhood with their Borough:

In [ ]:
Index = Canada['Neighbourhood'].isna()
print (Canada['Borough'].loc[Index]) # just one replacement
Canada['Neighbourhood'].loc[Index] = Canada['Borough'].loc[Index]

**Task:** *More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.*

Group by Postalcode, that means concenate all values of Neighbourhood per value of Postalcode:

In [ ]:
grouped = Canada.groupby(['Postalcode','Borough'])

rows = []

for name,group in grouped:
    #print( name)
    #print (group[['Postalcode','Borough']])
    row = [ name[0], name[1], (', '.join(group['Neighbourhood']) )]
    rows.append(row)

Canada = pd.DataFrame(rows, columns=headings)
pd.set_option('max_colwidth',140)
Canada

**Task:** *In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.*

In [ ]:
Canada.shape

Save DataFrame via object serialization with pickle.

In [ ]:
Canada.to_pickle("./Canada.pkl")